****CONTENT****

[SUPERVISED LEARNING](#1)
*  [CLASSIFICATIONS](#2) 
    * [K-Nearest Neighbors (KNN)](#3)
    * [Logistic Regression](#4)
    * [ROC Curve with Log Reg](#5)
    * [AUC (Area Under the ROC Curve)](#6)
    * [AUC using Cross Validation](#7)
    * [Confusion Matrix](#8)
*  [REGRESSIONS](#9)  
    * [Lınear Regression](#10)
    * [Cross Validation](#11)
    * [Regularized Regression](#12)
        * [Ridge Regression](#13)
        * [Lasso Regression](#14)  
* [Hyperparameter Tuning](#15)
    * [Grid Search CV](#16)
    * [Randomized Search CV](#17)
* [Preprocessing Data](#18)
    * [Dummy Variables](#19)
    * [Missing Values](#20)
        * [Imputer](#21)
        * [Pipeline](#22)  
    * [Normalizing (Scaling, or Centering)](#23)
    
        
        
    

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Supervised Learning**: uses labeled data

**Unsupervised Learning**: uses unlabeled data. uncovering hidden patterns and structures from unlabeled data.



<a id="1"></a> <br>
**SUPERVISED LEARNING**

Supervised learning'de, predictor variables (features)(independent variable) ve target variable (dependent variable) (response variable) vardır. 

Amaç, target variable'ı predictor variable'ları kullanarak tahmin edecek modeli oluşturmaktır.

**Not:**

Eğer **target** variable evet-hayır, spam-notspam gibi **kategori** biçiminde ise **CLASSIFICATION**,

Eğer **target** variable **continuously varying variable** ise, örneğin maaş, ev fiyatı gibi, **REGRESSION** uygulanır.

<a id="2"></a> <br>
**CLASSIFICATIONS**

<a id="3"></a> <br>
**K-Nearest Neighbors (KNN)**

In [ ]:
"""
Irıs data seti yükledik:
"""
iris = pd.read_csv('../input/iris-flower-dataset/IRIS.csv')

In [ ]:
"""
sepal_length, sepal_width, petal_length, petal_width are features (predictor variables)

species (setosa, versicolor, virginica) are target variables

"""

iris.head()

In [ ]:
"""
bunu belirtmek için target variable = y, predictor variables = X olarak ayırmamız gerekiyor.

"""

X = iris.drop('species', axis=1).values  
y = iris['species'].values

"""
.values diyerek modelin istediği formata (array) soktuk. 

"""

In [ ]:
"""
Basic Idea of KNN: k-closest data point'e bakarak unlabeled data point'in label'ını tahmin etme

"""

In [ ]:
#model importing

from sklearn.neighbors import KNeighborsClassifier #classifier ı import ettik.

#calling classifier with a variable

knn = KNeighborsClassifier(n_neighbors=6) #classifier'ı knn variable'ına atadık. 

In [ ]:
#train-test-split

#modele yerleştirilen datanın bir kısmı ile train yapılırken diğer bir kısmı ile de bu train
#test edilir. yani, örneğin, datanın %70'i ile optimum weight'ler belirlenir, kalan %30luk kısım ise bu modelin
#doğruluğu test edilir.

from sklearn.model_selection import train_test_split

"""
X_train: train data
X_test: test data
y_train: training labels
y_test: test labels

X:feature data, 
y:target, 
test_size:train-test oranı, 
random_state:aynı sonucu verir denediğimiz randomstate Id'si.
"""
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=21) # X ve y'yi kullanarak train-test yaptık.


In [ ]:
#.fit() 

knn.fit(X_train, y_train) #train data ile fit(yani train) yaptık. bu şu demek, datanın %70lik kısmının feature ve target(elimizde bilgi mevcut)
#valuelarını kullanarak modeli oluşturduk. 

In [ ]:
#.predict()

y_pred = knn.predict(X_test) #X_test (yani kalan %30luk feature datası) nı kullanarak bu feature dataya karşılık gelen prediction'ları oluşturduk
#buna da y_pred dedik. elimizde şu an y_pred ve y_test(gerçek değerler) mevcut. yani artık modelin performansını ölçebiliriz.b

In [ ]:
#.score()

knn.score(X_test, y_test) #score() ile modelin performansını ölçeriz. X_test'i verip, ondan elde edilen tahmin ile, y_test(gerçek değer)i kıyaslar.


In [ ]:
"""
Overfitting - Underfitting: finding optimum k-value

burada, manual olarak belirlenmesi gereken n_neighbors(k) parametresi mevcut. (hyperparameter)
k değeri küçük olursa overfitting yapmış olabiliriz.
büyüdükçe ise underfiting yapmış olabiliriz. 
optimum k değerini bulmamız gerekir!

"""

# Setup arrays to store train and test accuracies
neighbors = np.arange(1, 20)
train_accuracy = np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))

# Loop over different values of k
for i, k in enumerate(neighbors):
    # Setup a k-NN Classifier with k neighbors: knn
    knn = KNeighborsClassifier(n_neighbors=k)

    # Fit the classifier to the training data
    knn.fit(X_train, y_train)
    
    #Compute accuracy on the training set
    train_accuracy[i] = knn.score(X_train, y_train)

    #Compute accuracy on the testing set
    test_accuracy[i] = knn.score(X_test, y_test)

# Generate plot
plt.title('k-NN: Varying Number of Neighbors')
plt.plot(neighbors, test_accuracy, label = 'Testing Accuracy')
plt.plot(neighbors, train_accuracy, label = 'Training Accuracy')
plt.legend()
plt.xlabel('Number of Neighbors')
plt.ylabel('Accuracy')
plt.show()

#grafiğe göre, testing acuracy 2-3-4-5 aynı sonucu veriyor, sonra düşüş var. n=5 ideal seçim diyebiliriz.

<a id="4"></a> <br>
**Logistic Regression**

In [ ]:
"""
İsmi regression olabilir, ama classification'da kullanılır.

Mekanizmayı özetlemek gerekirse, 
Log Reg bize bir p,probability verir. p>0.5 ise target=1, p<0.5 ise target=0 diye sonuçlanır.
log reg bize linear decision boundary sağlar. 

"""

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
logreg = LogisticRegression()

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)

<a id="5"></a> <br>
**ROC Curve with Log Reg**

In [ ]:
"""
yukarıda bahsettiğimiz p-value default 0.5, ancak bu değeri değiştirirsek ne olur:
mesela, p=0.0 olursa, herşeyi 1 tahmin eder, p=1.0 olursa da herşeyi 0 tahmin edecektir.
bu p-value'ya threshold deriz, bu threshold'un değişiminin etkisini incelediğimiz curve'e ise
ROC (Receiver Operating Characteristic) Curve deriz.
Ancak, ROC curve binary classification için geçerlidir. bu yüzden heart disease dataseti kullanacağız:

"""

In [ ]:
df = pd.read_csv('../input/heart-disease-uci/heart.csv')

In [ ]:
df.head()

In [ ]:
X = df.drop('target', axis=1).values  
y = df['target'].values

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
logreg = LogisticRegression()

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.4, random_state=42)

In [ ]:
logreg.fit(X_train, y_train)

In [ ]:
y_pred = logreg.predict(X_test)

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
y_pred_prob = logreg.predict_proba(X_test)[:,1] #predict_proba 2 column döner, 1.si index olduğu için 2.columnı seçtik.

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob) #fpr:false pos.rate, tpr: true positive rate

In [ ]:
#grafiği çizelim:
plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr, tpr, label= 'Log Reg')
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('LogReg ROC Curve')
plt.show()

<a id="6"></a> <br>
**AUC (Area Under the ROC Curve)**

In [ ]:
""" Another metric for classification models.

ROC Curve'un altındaki alan (Area Under the ROC Curve - AUC) ne kadar fazla ise model o kadar iyi.

"""

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_test, y_pred_prob)

<a id="7"></a> <br>
**AUC using Cross Validation**

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
cv_scores = cross_val_score(logreg, X, y, cv=5, scoring='roc_auc') #scoring ile metodu belirledik.

In [ ]:
print(cv_scores)

<a id="8"></a> <br>
**Confusion Matrix**

In [ ]:
"""
Accuracy scores her zaman bize doğru bilgi vermez. çünkü;
örneğin, spam - not spam örneğinde, %99 real email, %1 spam iken hepsine real derse %99 doğru sonucu vermiş olur,
accuracy %99 olur. ama, bu bize modelin doğruluğu ile ilgili bilgiden ziyade, datanın içeriğinden kaynaklı bilgi vermiş olur.
yani, data imbalanced'tır. 

Confusion Matrix:
    Classification problemlerinde kullanılır. CM nedir:
    eğer email spam'se ve spam olarak tahmin edildiyse True Positive
    eğer email spam'se ve real olarak tahmin edildiyse False Negative
    eğer email real'se ve real olarak tahmin edildiyse True Negative
    eğer email real'se ve spam olarak tahmin edildiyse False Positive'dir.
    
    burada spam aradığımız için spam'e positive demiş olduk. bu bize bağlı.
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    precision = tp / (tp+fp)
    recall (sensitivity) = tp / (tp+fn)
    f1-score = 2 * precision * recall / (precision + recall)
    
    high preision: not many real emails predicted as spam
    high recall: predicted most spam emails correctly.
    

"""

In [ ]:
"""
yukarıda açıkladık zaten CM'i.
"""
#model import:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [ ]:
#yukarıdaki KNN modelini kullanarak Confusion Matrix'i örneklendirelim:
X = iris.drop('species', axis=1).values  
y = iris['species'].values

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=8) 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=21)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
y_pred = knn.predict(X_test)

In [ ]:
print(confusion_matrix(y_test, y_pred)) #-->>> ilk arguman true labellardan, ikinci argüman da predictionlardan oluştu.

In [ ]:
print(classification_report(y_test, y_pred))

<a id="9"></a> <br>
**REGRESSIONS**

Not:

Eğer **target** variable **continuously varying variable** ise, örneğin maaş, ev fiyatı gibi, **REGRESSION** uygulanır.

In [ ]:
boston = pd.read_csv('../input/bostoncsv/Boston.csv')

In [ ]:
boston.drop('Unnamed: 0', axis=1, inplace=True)
boston.head()

#target value medv'dir, diğerleri feature(predictor variable)

In [ ]:
# X ve y yi belirleyelim:

X = boston.drop('medv', axis=1).values 
y = boston['medv'].values

<a id="10"></a> <br>
**Lınear Regression**

In [ ]:
#linear regression, y=a+bx. more dimensional olunca y = a+bx1+cx2... 
#amaç, bu line öyle bir line olsun ki, residuals'ı(error unction)(loss or cost function) 
#(data point'lerin line'dan uzaklıkları(nın kareleri)) toplamı minimum olsun.
#y target, x feature, a ve b de bizim öğrenmek istediğimiz parametreler.
#elde ettiğimiz, residualsın karelerini minimize eden functiona da OLS (ORDİNARY Least Squares) denir.

In [ ]:
#sadece 1 column'ı alarak başlayalım; (one dimensional linear regression)
X_rooms = X[:,5]

In [ ]:
#shape'lere bakalım:

X_rooms.shape

In [ ]:
y.shape

In [ ]:
#shape'leri istenen formata getirelim:

y = y.reshape(-1,1)

In [ ]:
#istenen shape:

y.shape

In [ ]:
X_rooms = X_rooms.reshape(-1,1)

In [ ]:
X_rooms.shape

#tamamdır!..

In [ ]:
#bir grafikle iki variable'ı görelim:

plt.scatter(X_rooms, y)
plt.ylabel('value of house')
plt.xlabel('nr of rooms')

In [ ]:
#artık regression'a geldik:

from sklearn import linear_model #linear model'ı import ettik.


In [ ]:
reg = linear_model.LinearRegression() #modeli reg variable'ına atadık.

In [ ]:
reg.fit(X_rooms, y) #X_rooms ve y'ye göre fit ettik.

In [ ]:
#fitting line'ı grafik olarak göstermek için,
prediction_space = np.linspace(min(X_rooms), max(X_rooms)).reshape(-1,1)
plt.scatter(X_rooms, y)
plt.plot(prediction_space, reg.predict(prediction_space), color='blue')

In [ ]:
#ŞİMDİ, tüm datayı kullanarak more dimensional linear regression yapalım:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=21)

In [ ]:
reg_all = linear_model.LinearRegression()

reg_all.fit(X_train, y_train)

y_pred = reg_all.predict(X_test)

reg_all.score(X_test, y_test)

<a id="11"></a> <br>
**Cross Validation**

In [ ]:
"""
Model performansı datanın nasıl split edildiğiyle alakalı olabilir.
Bu yüzden, bulduğumuz model performans score aldatıcı olabilir.
Bu sorunu çözmek için cross validation metodu kullanılır.

İşleyişi şu şekildedir:
test_size=0.2 ise, CV ile, farklı 0.2'lik gruplara ayırarak modeli test eder.
önce ilk 0.2'yi test olarak alır, sonra 2. 0.2lik kısmı...
bu şekilde 5 farklı train-test grubuna göre performans belirlenir.
bu, 5-fold CV olarak addedilir. 

k-fold CV 'deki k değerinin optimumunu bulmamız gerekir. (k büyük olursa işlem süresi artar, 
k küçük olursa da geçerliliğini ölçemeyiz)

"""

In [ ]:
#import CV:
from sklearn.model_selection import cross_val_score 

In [ ]:
#üstteki dataya göre 5-fold cv:
cv_results = cross_val_score(reg_all, X, y, cv=5) 

In [ ]:
#sonuçlar:
cv_results

In [ ]:
# 5 farklı sonucun ortalaması:
np.mean(cv_results)

<a id="12"></a> <br>
**Regularized Regression**

In [ ]:
"""
Linear Regression ile loss function'ı minimize etmektir fitting line'ın amacı.
y=a1x1+a2x2+... 'daki a1,a2,ai lerin seçimi yapılır buna göre. 
buradaki coefficient'ların fazla büyük seçimi overfitting getirir.
multi dimensional linear regression'ı düşünürsek bu durum sağlıklı sonuç almayı engeller.
bunu aşmak için large coefficientları penalize eden bir yapı ile loss function kontrol edilebilir.
İşte buna regularization deniyor.

"""

In [ ]:
"""
Regularized Regression Types:

1. Ridge Regression 

(Loss function = OLS loss function + alfa* sum of squared values of each coefficient) 

(bu şekilde, Loss function coefficientlerin karelerinin toplamlarının alfa ile çarpımı kadar artmıştır. 
amaç ikinci kısmın da min olmasını sağlamaktır.)

(alpha, hyperparameter'dır.)

2. Lasso Regression

(Loss function = OLS loss function + alfa* sum of absolute values of each coefficient) !!!Tek fark bu!!!

!! Lasso regression, important feature seçimi yapılmakta kullanılır. Çünkü, küçük coefficientleri 0'a küçültme eğilimindedir.
0'a yuvarlanmamış feature'lar important olarak tanımlanabilir.

"""

<a id="13"></a> <br>
**Ridge Regression**

In [ ]:
#importing
from sklearn.linear_model import Ridge

In [ ]:
ridge = Ridge(alpha=0.1, normalize=True) #modeli oluşturduk. normalize=True tüm variable'lar aynı scale'da olması için.

In [ ]:
ridge.fit(X_train, y_train) #train ettik

In [ ]:
ridge_pred= ridge.predict(X_test)

In [ ]:
ridge.score(X_test, y_test)

<a id="14"></a> <br>
**Lasso Regression**

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso = Lasso(alpha=0.1, normalize=True)

In [ ]:
lasso.fit(X_train, y_train)

In [ ]:
lasso_pred = lasso.predict(X_test)

In [ ]:
lasso.score(X_test, y_test)

In [ ]:
#yukarıda bahsettiğimiz, önemsiz feature'ları 0 yapıp önemli olanları belirleme işlemi:
lasso_coef = lasso.fit(X, y).coef_

In [ ]:
lasso_coef

<a id="15"></a> <br>
**Hyperparameter Tuning**

<a id="16"></a> <br>
**Grid Search CV**

In [ ]:
"""
Modeldeki hyperparameterları belirterek grid search yaparak hyperparameterların optimum değerini bulmamızı sağlar.
"""

In [ ]:
iris = pd.read_csv('../input/iris-flower-dataset/IRIS.csv')

In [ ]:
X = iris.drop('species', axis=1).values  
y = iris['species'].values

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {'n_neighbors': np.arange(1,50)} #dict içinde '' içinde grid edilecek parametreyi yazıp, karşılığında da hangi değerlerin 
#deneneceğini yazdık.

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
knn_cv = GridSearchCV(knn, param_grid, cv=5) #this return the grid search object. fit etmeliyiz

In [ ]:
knn_cv.fit(X,y)

In [ ]:
knn_cv.best_params_

In [ ]:
knn_cv.best_score_

<a id="17"></a> <br>
**Randomized Search CV**

In [ ]:
"""
GridSearchCV'den farkı, çoklu hyperparameter belirlemede calculation yoğunluğu olduğu zaman kullanılmasıdır.
Tüm değerler denenmez, bir prob.dist kullanılarak hesaplama yapılır.

Bu uygulamayı yapmak için RandomForest kullanalım:
"""

In [ ]:
from scipy.stats import randint
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"max_depth": [3, None],
              "min_samples_leaf": randint(1, 9),
              "criterion": ["gini", "entropy"]}

In [ ]:
# Instantiate a Decision Tree classifier: tree
tree = DecisionTreeClassifier()

In [ ]:
# Instantiate the RandomizedSearchCV object: tree_cv
tree_cv = RandomizedSearchCV(tree, param_dist, cv=5)

In [ ]:
# Fit it to the data
tree_cv.fit(X, y)

In [ ]:
# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))

<a id="18"></a> <br>
**Preprocessing Data**

<a id="19"></a> <br>
**Dummy Variables**

In [ ]:
"""
categorical data type için, 



"""

In [ ]:
df = pd.read_csv('../input/iris-flower-dataset/IRIS.csv')

In [ ]:
df.head()

In [ ]:
df_species = pd.get_dummies(df)

In [ ]:
df_species.head()

In [ ]:
"""
3'ünden 2si değilse diğeridir. bu yüzden kullandığımız bir parametre var.
"""

In [ ]:
df_species2 = pd.get_dummies(df, drop_first=True)

In [ ]:
df_species2.head()

<a id="20"></a> <br>
**Missing Values**

<a id="21"></a> <br>
**Imputer**

In [ ]:
"""
bazen, missing value olarak görülmez, 0, '', vs vs olur.
bu durumda, hem nan value^ları smartly doldurmak için Imputer object kullanabiliriz.

"""

In [ ]:
df = pd.read_csv('../input/diabetescsv/diabetes.csv')

In [ ]:
df.head() #mesela burada, insulin = 0, bu not possible

In [ ]:
df.Insulin.replace(0, np.nan, inplace=True) #TÜM 0 ları nan value yaptık.
df.SkinThickness.replace(0, np.nan, inplace=True)
df.BMI.replace(0, np.nan, inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# df = df.dropna() # missing value olan tüm row'ları drop ettik.

In [ ]:
# df.shape #ancak, datanın yarısını kaybettik. unacceptible

In [ ]:
from sklearn.linear_model import LogisticRegression

#nan value'ların yerine başka şeyler koyabiliriz stratejiye bağlı olarak:
from sklearn.preprocessing import Imputer

In [ ]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0) #missing_values = 'Nan' olarak gösterimiş demek. axis=0 ile de sadece o column'a baktık.

In [ ]:
X = df.drop('Outcome', axis=1).values  
y = df['Outcome'].values

In [ ]:
imp.fit(X)

In [ ]:
X = imp.transform(X)

<a id="22"></a> <br>
**Pipeline**

In [ ]:
"""
Imputer'ın yaptığı işi yapar, üstüne bir de model çalıştırır.
"""

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression



In [ ]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
logreg = LogisticRegression()

In [ ]:
steps = [('imputation', imp), 
         ('logistic_regression', logreg)] #imputer modeli ve uygulanacak reg modelini steps e yazdık

In [ ]:
pipeline = Pipeline(steps)

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=21)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
pipeline.score(X_test, y_test)

<a id="23"></a> <br>
**Normalizing (Scaling, or Centering)**

In [ ]:
"""
Farklı feature'lar için farklı range'de değerlerin olması, ML modellerinin çalışmasını etkileyecektir.
Eğer scaling(normalizing) yapmazsak bazı feature'lar modelde daha ağırlıklı değerlendirilecek, 
sonuçlar yanıltıcı olacaktır.

"""

In [ ]:
iris = pd.read_csv('../input/iris-flower-dataset/IRIS.csv')

In [ ]:
iris.head()

In [ ]:
X =iris.drop('species', axis=1).values  
y = iris['species'].values

In [ ]:
from sklearn.preprocessing import scale

In [ ]:
X_scaled = scale(X)

In [ ]:
np.mean(X), np.std(X)

In [ ]:
np.mean(X_scaled), np.std(X_scaled)

In [ ]:
""" burada da, pipeline ile hem imputation, hem scaling, hem model çalışması,
    ardından da gridsearchcv ile hyperparameter optimization bir arada.
    
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Imputer
from sklearn.pipeline import Pipeline

# Setup the pipeline steps: steps
steps = [('imputation', Imputer(missing_values='NaN', strategy='mean', axis=0)),
         ('scaler', StandardScaler()),
         ('elasticnet', ElasticNet())]

# Create the pipeline: pipeline 
pipeline = Pipeline(steps)

# Specify the hyperparameter space
parameters = {'elasticnet__l1_ratio':np.linspace(0,1,30)}

# Create train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create the GridSearchCV object: gm_cv
gm_cv = GridSearchCV(pipeline, parameters)

# Fit to the training set
gm_cv.fit(X_train, y_train)

# Compute and print the metrics
print("Tuned ElasticNet Alpha: {}".format(gm_cv.best_params_))

"""